In [1]:
import random

import pandas as pd

In [2]:
file_device_id = "../data/input/Webメディア施策のABテスト・理論・具体例 - device.csv"
file_shop_id = "../data/input/Webメディア施策のABテスト・理論・具体例 - shop.csv"

In [3]:
df_device = pd.read_csv(file_device_id, header=None, names=["id", "group"])
df_shop = pd.read_csv(file_shop_id, header=None, names=["id"])

In [4]:
df_device_rows = df_device.shape[0]
df_shop_rows = df_shop.shape[0]

In [5]:
# ユニークなIDか
assert df_device["id"].nunique() == df_device_rows, (
    df_device["id"].nunique(),
    df_device_rows,
)
assert df_shop["id"].nunique() == df_shop_rows, (df_shop["id"].nunique(), df_shop_rows)

In [6]:
df_device_shop = df_device.merge(df_shop, how="cross", suffixes=("_device", "_shop"))

In [7]:
assert df_device_shop.shape[0] == df_device_rows * df_shop_rows

In [8]:
l_cv_flg = []
for i, row in df_device_shop.iterrows():
    # A群(groupが1の値)の場合は、55%の確率でCVすると仮定する。
    p = random.random()
    if (row["group"] == 1) & (p <= 0.55):
        l_cv_flg.append(1)
    elif (row["group"] == 1) & (p <= 0.55):
        l_cv_flg.append(0)
    # B群(groupが0の値)の場合は、50%の確率でCVすると仮定する。
    elif (row["group"] == 0) & (p <= 0.50):
        l_cv_flg.append(1)
    else:
        l_cv_flg.append(0)

In [9]:
df_device_shop["cv_flg"] = l_cv_flg

In [10]:
# A群の方がCVしやすいのか確認する
cv_num_per_g = df_device_shop.groupby("group").sum("cv_flg")["cv_flg"]
device_shop_num_per_g = df_device_shop.groupby("group").count()[
    "id_device"
]  # NOTE:この数値はdevice_idとid_shopの組み合わせ
cv_num_per_g / device_shop_num_per_g  # 確かにA群の方がCVしやすい

group
0    0.5178
1    0.5524
dtype: float64

In [12]:
df_device_shop.to_csv("../data/output/device_shop.csv", index=None)